In [1]:
from dotenv import load_dotenv
import os
import pickle

# Laden Sie die Umgebungsvariablen aus der .env-Datei
load_dotenv()

True

In [2]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import Chroma


loader = PyPDFLoader("./The-AI-Act.pdf")
pages = loader.load_and_split()




In [4]:
pages[5]

Document(page_content='EN 4  EN 1.2. Consistency with existing policy pr ovisions in the policy area  \nThe horizontal nature of the proposal requires full consistency with existing Union legislation \napplicable to sectors where high -risk AI systems are already used or likely to be used in the \nnear future.  \nConsistency is also ensu red with the EU Charter of Fundamental Rights and the existing \nsecondary Union legislation on data protection, consumer protection, non -discrimination and \ngender equality. The proposal is without prejudice and complements the General Data \nProtection Regula tion (Regulation (EU) 2016/679) and the Law Enforcement Directive \n(Directive (EU) 2016/680) with a set of harmonised rules applicable to the design, \ndevelopment and use of certain high -risk AI systems and restrictions on certain uses of remote \nbiometric id entification systems. Furthermore, the proposal complements existing Union law \non non -discrimination with specific requirements t

In [5]:
from langchain.evaluation.qa import QAGenerateChain
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [6]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in pages[:5]]
)

/Users/al02286558/anaconda3/lib/python3.10/site-packages/langchain/chains/llm.py:303: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [7]:
new_examples[0]

{'query': 'When was the proposal for the Regulation of the European Parliament and of the Council laying down harmonized rules on Artificial Intelligence (AI) presented?',
 'answer': 'The proposal for the Regulation of the European Parliament and of the Council laying down harmonized rules on Artificial Intelligence (AI) was presented on April 21, 2021.'}

In [12]:
examples = [
    {
        'query': 'When was the proposal for the Regulation of the European Parliament and of the Council laying down harmonized rules on Artificial Intelligence (AI) presented?',
        'answer': 'The proposal for the Regulation of the European Parliament and of the Council laying down harmonized rules on Artificial Intelligence (AI) was presented on April 21, 2021.'
    }
]

In [8]:
# Creating embeddings and Vectorization
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(pages,
                                    embedding=embeddings,
                                    persist_directory=".")
vectordb.persist()


In [10]:
from langchain.chains import RetrievalQA
llm = ChatOpenAI(temperature=0.9)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vectordb.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [13]:
examples += new_examples

In [14]:
qa.run(examples[0]["query"])



> Entering new  chain...

> Finished chain.


'The proposal for the Regulation of the European Parliament and of the Council laying down harmonized rules on Artificial Intelligence (AI) was presented on April 21, 2021.'

In [15]:
predictions = qa.apply(examples)



> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


LLM assisted evaluation¶

In [16]:
from langchain.evaluation.qa import QAEvalChain

In [17]:
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [18]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [19]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: When was the proposal for the Regulation of the European Parliament and of the Council laying down harmonized rules on Artificial Intelligence (AI) presented?
Real Answer: The proposal for the Regulation of the European Parliament and of the Council laying down harmonized rules on Artificial Intelligence (AI) was presented on April 21, 2021.
Predicted Answer: The proposal for the Regulation of the European Parliament and of the Council laying down harmonized rules on Artificial Intelligence (AI) was presented on April 21, 2021.
Predicted Grade: CORRECT

Example 1:
Question: When was the proposal for the Regulation of the European Parliament and of the Council laying down harmonized rules on Artificial Intelligence (AI) presented?
Real Answer: The proposal for the Regulation of the European Parliament and of the Council laying down harmonized rules on Artificial Intelligence (AI) was presented on April 21, 2021.
Predicted Answer: The proposal for the Regulation of t